In [33]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA

from tqdm.notebook import tqdm
from scipy.spatial.distance import cdist

# Load data

In [2]:
with open('../datasets/used_data/03_bert_like_models/02_topics.npy', 'rb') as f:
    df_topics = pd.DataFrame(np.load(f), columns=['topic'])
df_topics.shape

(6529, 1)

In [5]:
with open('../datasets/used_data/02_classical_ml/03_04_statements_herbert.npy', 'rb') as f:
    df_herbert = pd.DataFrame(np.load(f))
df_herbert.shape

(6529, 1024)

In [39]:
df = pd.read_parquet('../datasets/used_data/03_bert_like_models/01_basic_info_data.parquet')

df['labels'] = df['assestment'].astype(int)

df = df.sample(frac=1, random_state=111)

df = df[df.index.isin(df_topics.index)]
df.shape

(6529, 4)

# Process

In [8]:
cv_fold = []

for i in df_topics['topic'].unique().reshape(10,-1):
    train_cv = df_topics.index[ ~np.isin(df_topics["topic"], i) ].values
    test_cv = df_topics.index[ np.isin(df_topics["topic"], i) ].values
    
    cv_fold.append( [train_cv, test_cv])

# Example

## Run PCA on train data -> transform test

In [10]:
train_cv, test_cv = cv_fold[0]

In [24]:
# find n components allowing to have > 0.9 variance
n_com_list = []
for train_cv, test_cv in tqdm(cv_fold):
    data = df_herbert.loc[train_cv, :]
    
    pca = PCA()
    pca.fit(data)
    
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    
    n_comp = (cumulative_variance < 0.9).sum() + 1
    
    n_com_list.append(n_comp)

  0%|          | 0/10 [00:00<?, ?it/s]

In [26]:
n_com_list

[171, 171, 171, 172, 171, 171, 171, 171, 171, 171]

In [55]:
train_cv, test_cv = cv_fold[0]

data_train = df_herbert.loc[train_cv, :]
data_test = df_herbert.loc[test_cv, :]

df_train = df.loc[train_cv, :]
df_test = df.loc[test_cv, :]

pca = PCA(n_components = 171)
pca.fit(data_train)

data_train_pca = pd.DataFrame(pca.transform(data_train), index=train_cv)
data_test_pca = pd.DataFrame(pca.transform(data_test), index=test_cv)

data_train_pca_fake = data_train_pca[df_train['assestment']==1]
data_train_pca_true = data_train_pca[df_train['assestment']==0]

df_train_fake = df_train[df_train['assestment']==1]
df_train_true = df_train[df_train['assestment']==0]

In [56]:
def get_3_nearest(example, space):
    distances = cdist(example, space, 'euclidean')

    nearest_indices = np.argsort(distances)[0][:3]
    print("Indices of the three nearest neighbors:", nearest_indices)
    print("Distances to the three nearest neighbors:", distances[0, nearest_indices])

    return nearest_indices

In [58]:
example = data_test_pca.iloc[0,:].values.reshape(1,-1)
nearest_fake = get_3_nearest(example, data_train_pca_fake)
nearest_true = get_3_nearest(example, data_train_pca_true)

Indices of the three nearest neighbors: [ 240 2842 1485]
Distances to the three nearest neighbors: [3.80331861 4.05540636 4.23394135]
Indices of the three nearest neighbors: [   3 1614   27]
Distances to the three nearest neighbors: [3.22325752 3.57858447 3.68345603]


In [60]:
df_test[['assestment', 'text']].head(1).values

array([[0,
        '(...) udzielenie bankowi centralnemu Ukrainy tzw. swapa walutowego w kwocie 1 mld dolarów. Narodowy Bank Polski był pierwszym i jak dotąd jedynym bankiem centralnym, który zaoferował Narodowemu Bankowi Ukrainy takie wsparcie.']],
      dtype=object)

In [63]:
df_train_fake[['assestment', 'text']].iloc[nearest_fake, :].values

array([[1,
        '70 miliardów euro długu państwowego jest denominowane w walutach obcych.'],
       [1,
        'To byłaby kompletna ruina w tej chwili budżetu państwa, czyli doprowadzilibyśmy w prosty sposób do niewypłacalności Polski'],
       [1,
        'A w Polsce potrzebujemy takich rozwiązań jakie zostały wprowadzane w Rumunii - 102 podatki i opłaty zostały przez, notabene lewicowy, rząd zlikwidowane. I dziś Rumunia ma najwyższy wskaźnik wzrostu gospodarczego w całej Unii Europejskiej.']],
      dtype=object)

In [65]:
df_train_true[['assestment', 'text']].iloc[nearest_true, :].values

array([[0,
        'W listopadzie tamtego roku pan prezydent Andrzej Duda przyznał specjalną nagrodę gospodarczą panu Glapińskiemu, za utrzymaniu stabilności bankowej.'],
       [0,
        '[Węgry] wprowadziły podatek od aktywów bankowych już w 2010 r. Od tego czasu akcja kredytowa na Węgrzech zmalała prawie o 1/3.'],
       [0,
        'Finansowy rynek Rosji jest w tej chwili zdominowany (...) w 55 proc, przez 4 główne banki. ']],
      dtype=object)